In [ ]:
import os
base_url = os.environ["API_BASE"]
print(base_url)

In [ ]:
latitude = 65.1
longitude = -146.6

In [ ]:
import micropip
await micropip.install('pyodide-http')
import pyodide_http
pyodide_http.patch_all()

In [ ]:
import pandas as pd
df = pd.read_csv(f"{base_url}degree_days/thawing_index/{latitude}/{longitude}?format=csv", header=3)
df

In [ ]:
await micropip.install("ipywidgets")
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# function to compute design thawing index
def compute_and_display_design_index(start_year, end_year, n_warmest_years):
    
    df_slice = df[(df["year"] >= start_year) & (df["year"] < end_year)]
    design_index = round(df_slice.sort_values("dd", ascending=False).iloc[0:n_warmest_years]["dd"].mean())
    return design_index

In [ ]:
# create interactive widgets for input
start_year_slider = widgets.IntSlider(
    value=2030, min=1980, max=2099, step=1, description="Start Year:"
)
end_year_slider = widgets.IntSlider(
    value=2060, min=1981, max=2100, step=1, description="End Year:"
)
n_coldest_years_slider = widgets.IntSlider(
    value=3, min=1, max=10, step=1, description="Number of Warmest Years:",
    style={"description_width": "initial"}
)

# create an output widget for dynamic displaying of result
output = widgets.Output()

# function to handle widget changes
def handle_slider_change(change):
    with output:
        output.clear_output()

        if (
            end_year_slider.value - start_year_slider.value
            < n_coldest_years_slider.value
        ):
            start_year_slider.value = (
                end_year_slider.value - n_coldest_years_slider.value
            )
            output.clear_output()
        result = compute_and_display_design_index(
            start_year_slider.value, end_year_slider.value, n_coldest_years_slider.value
        )
        print(f"Design Thawing Index: {result} °F•days")

# attach the change handler to the sliders
start_year_slider.observe(handle_slider_change, names="value")
end_year_slider.observe(handle_slider_change, names="value")
n_coldest_years_slider.observe(handle_slider_change, names="value")

handle_slider_change(None)

# display the widgets with the title and initial result
display(start_year_slider, end_year_slider, n_coldest_years_slider, output)
initial_result = compute_and_display_design_index(
    start_year_slider.value, end_year_slider.value, n_coldest_years_slider.value
)
